In [1]:
import cv2
import numpy as np
import imutils
import os
import glob
from keras.preprocessing.image import img_to_array
import tensorflow as tf
from tensorflow import keras
# Local experiment

In [2]:
def getmodel():
    model =keras.models.load_model('../model/keras/best_model.hdf5')
    return model

def preprocess_frame(data,img_size):
    data = imutils.resize(data, width=img_size)
    grey = cv2.cvtColor(data,cv2.COLOR_BGR2GRAY)

    return data,grey

def detectface(frame,scale=1.05, Neigh=11, mSize=(20, 20)):
    detector = cv2.CascadeClassifier('../model/haarcascade/haarcascade_frontalface_default.xml')
    return detector.detectMultiScale(frame, scaleFactor=scale, minNeighbors=Neigh, minSize=mSize,flags=cv2.CASCADE_SCALE_IMAGE)


def process_data(data,img_height=255,img_size=400):

    EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

    frame,grey = preprocess_frame(data,img_size)
    canvas = np.zeros((img_height, img_size, 3), dtype="uint8")
    frameClone = frame.copy()
    rects = detectface(grey)
    model = getmodel()
    if len(rects) > 0:
        rect = sorted(rects, reverse=False,key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]

        (fX, fY, fW, fH) = rect 

        roi = grey[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        preds = model.predict(roi)[0]
        label = EMOTIONS[preds.argmax()]

        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
            # construct the label text
            text = "{}: {:.2f}%".format(emotion, prob * 100)

            # draw the label + probability bar on the canvas
            w = int(prob * img_size)
            cv2.rectangle(canvas, (5, (i * 35) + 5),
            (w, (i * 35) + 35), (0, 0, 255), -1)
            cv2.putText(canvas, text, (10, (i * 35) + 23),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45,
            (255, 255, 255), 2)
        
        text_all = f'{label} {preds.max()*100:.2f}%'

        cv2.putText(frameClone, text_all, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),(0, 0, 255), 2)

    else:
        label = 'Notfound'
    
    return frameClone,canvas,label
    

In [7]:
def getdata(type,path=None):

    img_size=400
    img_height=225

    # Input is an image
    if type =='image':
        path_list  = glob.glob('../data/image/*.jpg')
        for path in path_list:
            data = cv2.imread(path)

            pred_data,canvas,label = process_data(data,img_height,img_size)

            cv2.imshow("Face", pred_data)
            cv2.imshow("Probabilities", canvas)

            cv2.waitKey(0) 
            cv2.destroyAllWindows()

            filename = path.split(os.path.sep)[-1]
            filename = label + '_' + filename
            cv2.imwrite(filename=f'../output/img/{filename}',img=pred_data)
            
    
    # Input is  video
    else:

        if type =='video':
            path_list = glob.glob('../data/video/*.mp4')
            path = path_list[0]

            filename = path.split(os.path.sep)[-1]

            fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # You can choose a different codec based on your preference
            output_video = cv2.VideoWriter(f'../output/video/output_{filename}', fourcc, 30.0, (img_size,img_height))            

            camera = cv2.VideoCapture(path)

        else:
            filename = 'live'
            fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
            output_video = cv2.VideoWriter(f'../output/video/output_{filename}.avi', fourcc, 30.0, (img_size,img_height))

            camera = cv2.VideoCapture(0)
       

        while True:
            grabbed,data = camera.read()

            if not grabbed:
                print('End of Video...')
                break

            pred_data,canvas,label = process_data(data,img_height,img_size)

            cv2.imshow("Face", pred_data)
            cv2.imshow("Probabilities", canvas)

            output_video.write(pred_data)
            
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        camera.release()
        output_video.release()

        cv2.dqestroyAllWindows()



In [9]:
type = ['image','video','live']
getdata(type[2],path=None)